In [31]:

import requests 
from bs4 import BeautifulSoup
import csv 
import selenium 
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta
from datetime import date
import time

# Using Chrome to access web
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def tee_time_Search(number_of_golfers,desired_golf_date):
    # dictionary of courses I like to play, or want to play
    courses = {
        'Glen_club':'https://theglenclub.ezlinksgolf.com/index.html#/search',
        'Thunderhawk': 'https://thunderhawk.ezlinksgolf.com/index.html#/search',
        'Harborside': 'https://harborsidepp.ezlinksgolf.com/index.html#/search',
        'Ruffled Feathers': 'https://ruffledfeatherspp.ezlinksgolf.com/index.html#/search?utm_source=google&utm_medium=organic&_ga=2.206285780.678145072.1600650543-1657383022.1600650543',
        'Bollingbrook Golf Club': 'https://bolingbrookpp.ezlinksgolf.com/index.html#/search',
        'Sunset Valley':'https://sunsetvalley.ezlinksgolf.com/index.html#/search',
        'Stonewall Orchard':'https://stonewall-orchard-golf-club.book.teeitup.golf/',
        'Mount Prospect Golf Club': 'https://mt-prospect-golf-club.book.teeitup.golf/?inline#!xd=https%3A%2F%2Fwww.golfmtprospect.com%2Ftee-times%2F',
        'Cantigny Golf Club': 'https://cantignygolf.ezlinksgolf.com/index.html#/search',
        'Ravisloe Golf Club': 'https://ravisloe-country-club.book.teeitup.golf/?',
        'Deerfield': 'https://deerfieldpp.ezlinksgolf.com/index.html#/search',
        'Countryside': 'https://countryside.ezlinksgolf.com/index.html#/search',
        'Eaglewood':'https://eaglewood-resort-and-spa.book.teeitup.golf/'
    }

    # Courses generally have 1 of two formats - "EZ Links" or "Tee It Up"




    ez_links_golf = [
         'Glen_club',
         'Thunderhawk',
         'Harborside',
         'Ruffled Feathers',
         'Bollingbrook Golf Club',
         'Sunset Valley',
         'Cantigny Golf Club',
         'Deerfield',
         'Countryside'

    ]


    tee_it_up = ['Stonewall Orchard','Mount Prospect Golf Club','Ravisloe Golf Club','Eaglewood']


    golf_date = str(desired_golf_date)
    golf_date_day = int(golf_date.split('/')[1])
    current_date = datetime.today().date()
    current_day = datetime.today().day
    golfers = int(number_of_golfers)


    #check date
    try:
        date_format = datetime.strptime(golf_date,'%m/%d/%Y').date()
    except:
        return "Please enter the date in MM/DD/YYYY format."

    if ((date_format < current_date) | (date_format > (current_date + timedelta(days=7)))) :
        return "Please enter a date within 7 days of today."
    if ((golfers < 0) | (golfers > 4)):
        return "Please enter a valid number of golfers (1,2,3 or 4 players)"


    final_times = {}

    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    # options.add_argument('--incognito')
    options.add_argument("--window-size=1920,1080")
    options.add_argument('--headless')


    driver = webdriver.Chrome(ChromeDriverManager().install(),options = options)
    driver.implicitly_wait(10)
    # Open the website
    #begin loop
    for course in courses:
        if course in ez_links_golf:
            time_list = []
            try:
                driver.get(courses[course])

                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, "pickerDate")))
                date_table = driver.find_element_by_id("pickerDate")
                date_table.clear()

                driver.implicitly_wait(5)
                date_table.send_keys(str(golf_date))

                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, "ui-state-active")))
                date_ez = driver.find_element_by_class_name("ui-state-active")
                date_ez.click()

                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, "players-button")))
                num_players = driver.find_element(By.ID,"players-button")
                num_players.click()


                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.LINK_TEXT, str(golfers))))
                golfers_button = driver.find_element_by_link_text(str(golfers))
                golfers_button.click()

                final_page = driver.page_source
                # we now have a list of times for the course, date, and number of golfers we want - now use BS to return values


                ################################
                    #Try alternative way of getting times
                ################################
                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//ul[@class='tee-time-block']")))
                tee_time_block = driver.find_element(By.XPATH,"//ul[@class='tee-time-block']")
                tee_time_block_list = tee_time_block.find_elements(By.XPATH,".//li")
                for tee_time_card in tee_time_block_list:
                    driver.implicitly_wait(10)
                    tee_time = tee_time_card.find_element(By.XPATH,"./span").text
                    #print(tee_time)
                    time_list.append(tee_time)
            except:
                time_list = ['Website Timeout - did not load correctly']
            final_times[course] = time_list
        elif course in tee_it_up:
            try:
                time_list = []
                driver.get(courses[course])
                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[@id='app']/div/section/div/div/div/div[1]/div/div[2]/div/div/div[3]/div/div[2]/button")))


                date1 = driver.find_element(By.XPATH, "//*[@id='app']/div/section/div/div/div/div[1]/div/div[2]/div/div/div[3]/div/div[2]/button")
                date1.click()
                # check if same month or next month
                #if next month
                if golf_date_day < current_day: 
                    #click next month arrow
                    WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[@id='app-body']/div[2]/div[2]/div[1]/div[2]/div[1]/button[2]")))
                    next_month_arrow = driver.find_element(By.XPATH, "//*[@id='app-body']/div[2]/div[2]/div[1]/div[2]/div[1]/button[2]")
                    next_month_arrow.click()
                    driver.implicitly_wait(5)

                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH,"//div[@role='document']")))  
                calendar = driver.find_element(By.XPATH,"//div[@role='document']")
                list_dates = calendar.find_elements(By.XPATH,".//div[@role='presentation']")
                for date in list_dates:
                    date_num = date.find_element(By.XPATH,".//button/span[1]").text
                    if date_num == str(golf_date_day):
                        #print("Found Date")
                        correct_date = date.find_element(By.XPATH,".//button")
                correct_date.click()

                #click ok
                driver.find_element(By.XPATH,"//*[@id='app-body']/div[2]/div[2]/div[2]/button[3]").click()

                #wait for number of golfers to load and click
                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH,"//button[@value='{}']".format(str(golfers)))))
                driver.find_element(By.XPATH,"//button[@value='{}']".format(str(golfers))).click()
                #convert to beautiful soup and return times
                WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH,'//*[@id="app"]/div/section/div/div/div/div[2]/div/div[6]/section/div')))
                driver.find_element(By.XPATH,'//*[@id="app"]/div/section/div/div/div/div[2]/div/div[6]/section/div');
                soup = BeautifulSoup(driver.page_source)
                time_boxes = soup.find_all('h3')

                for time in time_boxes:
                    if ':' in time.get_text() and 'Sort' not in time.get_text():
                        time_list.append(time.get_text())
            except:
                time_list = ['Error Getting Times']
            final_times[course] = time_list
    for course_times in final_times:
        print(course_times + ':',final_times[course_times])
        print('\n')
    # driver.quit()
    return
